In [125]:
# Estos dos comandos evitan que haya que hacer reload cada vez que se modifica un paquete
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Anotaciones:
Desde el lado de Estadistica querriamos ver que tan bueno se aproxima a una gaussiana. Desde ML que tan bien generaliza

In [126]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from collections import Counter
from mpl_toolkits.mplot3d import Axes3D

In [127]:
N = -1

In [128]:
data = pd.read_csv('data/alturas-pesos-mils-train.csv')[:N]

In [129]:
test = pd.read_csv('data/alturas-pesos-mils-test.csv')

In [130]:
from BiVariateJoint import BiVariateJoint

In [131]:
data_hombres = data.loc[data['Genero']=='Hombre'][['Peso','Altura']].values
data_mujeres = data.loc[data['Genero']=='Mujer'][['Peso','Altura']].values

# Grafico los datos

In [101]:
%matplotlib qt
f = plt.figure(figsize=(20,10))
plt.scatter(data_hombres[:,0], data_hombres[:,1], color='b', s=2, label='Hombres')
plt.scatter(data_mujeres[:,0], data_mujeres[:,1], color='r', s=2, label='Mujeres')
plt.xlabel('Pesos [cms]')
plt.ylabel('Alturas [cms]')
plt.legend()
plt.show()

# Regla de Bayes (Continuo)
\begin{equation}
P(Hombre | X_1, X_2) = \frac{p(X_1,X_2 | Hombre) P(Hombre)}{p(X_1,X_2)} \\ \\
\end{equation}

\begin{equation}
P(Mujer | X_1, X_2) = \frac{p(X_1,X_2 | Mujer) P(Mujer)}{p(X_1,X_2)}
\end{equation}

Que tenemos?
\begin{equation}
P(Hombre) = \frac{N_{H}}{N_H + N_M} \quad
\end{equation}

\begin{equation}
P(Mujer) = \frac{N_{M}}{N_H + N_M} \quad
\end{equation}


\begin{equation}
\large
p(X_1,X_2) = p(X_1,X_2 | Mujer) P(Mujer) + p(X_1,X_2 | Hombre) P(Hombre)
\end{equation}

Con estimar:
\begin{equation}
P(X_1,X_2 | Mujer) \quad y \quad
P(X_1,X_2 | Hombre) 
\end{equation}
\begin{equation}
o
\end{equation}
\begin{equation}
p(X_1,X_2 | Mujer) \quad y \quad
p(X_1,X_2 | Hombre) 
\end{equation}

# Bayes Gaussiano (Quadratic Discriminant Analisys)

\begin{equation}
P(Hombre | X_1, X_2) = \frac{p(X_1,X_2 | Hombre) P(Hombre)}{p(X_1,X_2)} \\
P(Mujer | X_1, X_2) = \frac{p(X_1,X_2 | Mujer) P(Mujer)}{p(X_1,X_2)} \\
p(X_1,X_2) = p(X_1,X_2 | Hombre) P(Hombre) + p(X_1,X_2 | Mujer) P(Mujer)
\end{equation}

\begin{equation}
P(C_0 | X_1, X_2)  \rightarrow p \\
P(C_1 | X_1, X_2) \rightarrow (1-p) \\
\end{equation}
### Log-odds
\begin{equation}
\frac{p}{1-p} = \frac{P(X_1,X_2 | C_0) P(C_0)}{
 P(X_1,X_2 | C_1) P(C_1)}
\end{equation}

In [132]:
peso_altura_hombres = data.loc[data['Genero']=='Hombre'][['Peso', 'Altura']].values
peso_altura_mujeres = data.loc[data['Genero']=='Mujer'][['Peso', 'Altura']].values

In [133]:
from scipy.stats import multivariate_normal

In [134]:
def get_gauss_prob(data, mean_hombres, mean_mujeres, cov_hombres, cov_mujeres):
    data_np = data
    likelihood_class_1 = multivariate_normal.pdf(data_np, mean_hombres, cov_hombres)
    likelihood_class_2 = multivariate_normal.pdf(data_np, mean_mujeres, cov_mujeres)
    N_class_1 = len(peso_altura_hombres)
    N_class_2 = len(peso_altura_mujeres)
    prior_1 = N_class_1/(N_class_1 + N_class_2)
    prior_2 = N_class_2/(N_class_1 + N_class_2)
    total = likelihood_class_1 * prior_1 + likelihood_class_2 * prior_2
    p_class_1 = likelihood_class_1 * prior_1/total
    p_class_2 = likelihood_class_2 * prior_2/total
    return p_class_1, p_class_2

In [135]:
def get_acc_gauss(data, mean_hombres, mean_mujeres, cov_hombres, cov_mujeres):
    p_class_1, p_class_2 = get_gauss_prob(data[['Peso', 'Altura']].values, mean_hombres, mean_mujeres, cov_hombres, cov_mujeres)
    return ((p_class_1>p_class_2)==(data['Genero']=='Hombre')).sum()/len(p_class_1)

# LDA

### Supone $\Sigma_0=\Sigma_1$

In [136]:
mean_hombres = peso_altura_hombres.mean(axis=0)
mean_mujeres = peso_altura_mujeres.mean(axis=0)
cov_LDA = np.cov(np.vstack([peso_altura_hombres - mean_hombres, peso_altura_mujeres - mean_mujeres]).T)

In [137]:
print(cov_LDA)
print(np.cov(peso_altura_hombres.T)) 
print(np.cov(peso_altura_mujeres.T))

[[77.94597261 53.52970001]
 [53.52970001 50.2964779 ]]
[[80.71054209 56.29362173]
 [56.29362173 53.03283768]]
[[75.19882927 50.77709625]
 [50.77709625 47.57064786]]


In [138]:
acc_train_LDA =  get_acc_gauss(data, mean_hombres, mean_mujeres, cov_LDA, cov_LDA)
print(acc_train_LDA)

0.9167395924490561


In [139]:
acc_test_LDA = get_acc_gauss(test, mean_hombres, mean_mujeres, cov_LDA, cov_LDA)
print(acc_test_LDA)

0.9225


In [140]:
N = 300
X = np.linspace(data.min()['Peso'], data.max()['Peso'], N)
Y = np.linspace(data.min()['Altura'], data.max()['Altura'], N)
X, Y = np.meshgrid(X, Y)

# Pack X and Y into a single 3-dimensional array
pos = np.empty(X.shape + (2,))
pos[:, :, 0] = X
pos[:, :, 1] = Y

# The distribution on the variables X, Y packed into pos.
Z_H_LDA = multivariate_normal.pdf(pos, mean_hombres, cov_LDA)
Z_M_LDA = multivariate_normal.pdf(pos, mean_mujeres, cov_LDA)
Z_LDA, _ = get_gauss_prob(pos, mean_hombres, mean_mujeres, cov_LDA, cov_LDA)

In [142]:
%matplotlib qt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
fig = plt.figure(figsize=(20,10))
ax = fig.gca(projection='3d')
cm = plt.cm.RdBu
#cf = ax.contourf(X, Y, Z, 256, alpha=.8, vmin=0., vmax=1., cmap=cm)
#plt.colorbar(cf, ax=ax)
ax.contourf(X, Y, Z_H_LDA, 256)
ax.contourf(X, Y, Z_M_LDA, 256)
ax.view_init(70, -90)
ax.set_ylabel('Alturas [cms]')
ax.set_xlabel('Pesos [kgs]')
plt.show()

### Log-Odds

### Partiendo de Bayes (QDA)

\begin{equation}
\large
-\frac{1}{2}(\vec{x}-\vec{\mu}_0)^T\Sigma_0^{-1}(\vec{x}-\vec{\mu}_0) + \frac{1}{2}(\vec{x}-\vec{\mu}_1)^T\Sigma_1^{-1}(\vec{x}-\vec{\mu}_1)  - ln(|\Sigma_0|) + ln(|\Sigma_1|) + log(\frac{P(C_0)}{P(C_1)}) = 0
\end{equation}

### Suponiendo $\Sigma_0=\Sigma_1=\Sigma$
### Como las matrices de covarianza son diagonales:  $\quad \vec{x}^T\Sigma^{-1}\vec{\mu}_i = \vec{\mu}_i^T\Sigma^{-1}\vec{x}$

\begin{equation}
\large
-\frac{1}{2}(\vec{x}-\vec{\mu}_0)^T\Sigma^{-1}(\vec{x}-\vec{\mu}_0) + \frac{1}{2}(\vec{x}-\vec{\mu}_1)^T\Sigma^{-1}(\vec{x}-\vec{\mu}_1) + log(\frac{P(C_0)}{P(C_1)}) = 0
\end{equation}

\begin{equation}
\large
-\frac{1}{2}(\vec{x}^T\Sigma^{-1}\vec{x}+\vec{\mu}_0^T\Sigma^{-1}\vec{\mu}_0-\vec{\mu}_0^T\Sigma^{-1}\vec{x}-\vec{x}^T\Sigma^{-1}\vec{\mu}_0) + \frac{1}{2}(\vec{x}^T\Sigma^{-1}\vec{x}+\vec{\mu}_1^T\Sigma^{-1}\vec{\mu}_1-\vec{\mu}_1^T\Sigma^{-1}\vec{x}-\vec{x}^T\Sigma^{-1}\vec{\mu}_1) + log(\frac{P(C_0)}{P(C_1)}) = 0
\end{equation}

\begin{equation}
\large
-\frac{1}{2}(\vec{\mu}_0^T\Sigma^{-1}\vec{\mu}_0-2\vec{\mu}_0^T\Sigma^{-1}\vec{x}) + \frac{1}{2}(\vec{\mu}_1^T\Sigma^{-1}\vec{\mu}_1-2\vec{\mu}_1^T\Sigma^{-1}\vec{x}) + log(\frac{P(C_0)}{P(C_1)}) = 0
\end{equation}

\begin{equation}
\large
-\frac{1}{2}(\vec{\mu}_0+\vec{\mu}_1)^T\Sigma^{-1}(\vec{\mu}_0-\vec{\mu}_1)+\vec{x} ^T\Sigma^{-1}(\vec{\mu}_0- \vec{\mu}_1) + log(\frac{P(C_0)}{P(C_1)}) = 0
\end{equation}


\begin{equation}
\large
X^TW + C = 0
\end{equation}

\begin{equation}
\large
W = \Sigma^{-1}(\vec \mu_0- \vec \mu_1)
\end{equation}

\begin{equation}
\large
c =  log(\frac{P(C_0)}{P(C_1)}) - \frac{1}{2}(\vec{\mu}_0+\vec{\mu}_1)^T\Sigma^{-1}(\vec{\mu}_0-\vec{\mu}_1) = log(\frac{P(C_0)}{P(C_1)}) - \frac{1}{2}(\vec{\mu}_0+\vec{\mu}_1)^T W
\end{equation}

In [147]:
%matplotlib qt
fig = plt.figure(figsize=(20,10))
ax = fig.gca(projection='3d')
cm = plt.cm.RdBu
ax.contourf(X, Y, Z_LDA, 256, vmin=0., vmax=1., cmap=cm)
ax.set_title('TRAIN: ' + str(np.round(acc_train_LDA*100)/100) + ' - CV:' + str(np.round(acc_test_LDA*100)/100))
ax.set_ylabel('Alturas [cms]')
ax.set_xlabel('Pesos [kgs]')
ax.view_init(70, -90)
plt.show()

In [143]:
W = np.linalg.inv(cov_LDA).dot(mean_hombres - mean_mujeres)
C = np.log(peso_altura_hombres.shape[0]/peso_altura_mujeres.shape[0]) - 0.5*(mean_hombres + mean_mujeres).T.dot(np.linalg.inv(cov_LDA)).dot(mean_hombres - mean_mujeres) 

In [144]:
# Verificación ecuación
z_teorica = np.zeros([N, N])
for row in range(pos.shape[0]):
    for col in range(pos.shape[1]):
        z_teorica[row, col] = pos[row, col].T.dot(W) + C

In [108]:
# Verificación
%matplotlib qt
fig = plt.figure(figsize=(20,10))
ax = fig.gca()
cm = plt.cm.RdBu
cf = ax.contourf(X, Y, Z_LDA, 256, alpha=.8, vmin=0., vmax=1., cmap=cm)
plt.colorbar(cf, ax=ax)
z_levels = np.logspace(-5,-2,10)/4
#ax.contour(X, Y, Z_H, z_levels)
#ax.contour(X, Y, Z_M, z_levels)
ax.scatter(data_hombres[:,0], data_hombres[:,1], color='b', s=2, label='Hombres')
ax.scatter(data_mujeres[:,0], data_mujeres[:,1], color='r', s=2, label='Mujeres')
ax.contour(X, Y, Z_LDA, (0.5,), colors='k', linewidths=0.25)
#ax.contour(X, Y, z_teorica, (0,), colors='k', linewidths=0.25)
ax.set_ylabel('Alturas [cms]')
ax.set_xlabel('Pesos [kgs]')
plt.show()

In [146]:
%matplotlib qt
fig = plt.figure(figsize=(20,10))
ax = fig.gca()
cm = plt.cm.RdBu
cf = ax.contourf(X, Y, Z_LDA, 256, alpha=.8, vmin=0., vmax=1., cmap=cm)
plt.colorbar(cf, ax=ax)
z_levels = np.logspace(-5,-2,5)/4
ax.contour(X, Y, Z_H_LDA, z_levels, alpha=0.5)
ax.contour(X, Y, Z_M_LDA, z_levels, alpha=0.5)
ax.contour(X, Y, Z_LDA, (0.5,), colors='k', linewidths=1)
ax.scatter(data_hombres[:,0], data_hombres[:,1], color='b', s=1, label='Hombres')
ax.scatter(data_mujeres[:,0], data_mujeres[:,1], color='r', s=1, label='Mujeres')
ax.set_title('TRAIN: ' + str(np.round(acc_train_LDA*100)/100) + ' - CV:' + str(np.round(acc_test_LDA*100)/100))
ax.set_ylabel('Alturas [cms]')
ax.set_xlabel('Pesos [kgs]')
plt.legend()
plt.show()

|Modelo| tipo| Train Acc| CV Acc|Comentarios
|-| -| -| -|-|
|Histogram - step 0.25|Bayes|0.97|0.68
|Histogram - step 0.25|Naive Bayes|0.89|0.88
|Histogram - step 0.5|Bayes|0.94|0.82
|Histogram - step 0.5|Naive Bayes|0.89|0.88
|Histogram - step 1|Bayes|0.92|0.90
|Histogram - step 1|Naive Bayes|0.89|0.88
|Histogram - step 4|Bayes|0.91|0.92| No generaliza por afuera de la zona de los puntos
|Histogram - step 4|Naive Bayes|0.89|0.88
|Histogram - step 8|Bayes|0.90|0.90
|Histogram - step 8|Naive Bayes|0.87|0.86
|Gaussian|Bayes|0.92|0.92|
|Gaussian|Naive Bayes|0.89|0.89|
|Regresion Logística|-|0.92|0.92|